### Project 3

## Part of the Quantum Computing Bootcamp at the Erdos Institute.

The goal of this project is to implement a quantum calculator for $d$-qubit numbers using qiskit.
There are two main objectives:
1. Implement addition modulo $2^d$;
2. Implement multiplication modulo $2^d$.

These objectives are subject to the following constraints:
- Two $d$-qubit registers, $|x\rangle$ and $|y\rangle$, holding the two input numbers;
- One control qubit, $|z \rangle$, that implements addition if $z = 0$ and multiplication if $z = 1$;
- One $d$-qubit output register, which holds the result of the chosen operation;
- No gates other than arbitrary 1-qubit, control not, and Toffoli gates may be used

Finally, an analysis of the complexities and resources used is to be added at the end of the project.

---

Satisfying the above objectives and constraints would complete the project, but to make our calculator more useful, we would like to implement more operations.
As such, we list the following in order of implementation, where a strikethrough indicates that the operation was added successfully:
- Implement subtraction modulo $2^d$
- Implement modular exponentiation modulo $2^d$


In [33]:
from qiskit.circuit import QuantumCircuit, QuantumRegister, AncillaRegister, Parameter
from qiskit.quantum_info import Statevector, Operator

import numpy as np

### Parameters go here
d = 2

### Load registers

x_register = QuantumRegister(size=d, name="x")
y_register = QuantumRegister(size=d, name="y")
z_register = QuantumRegister(size=1, name="z")
output_register = AncillaRegister(size=d, name="w")

## Circuits necessary for the calculator below here
---
As we are only allowed CX and Toffoli gates for dealing with more than one qubit, we need to decompose the controlled phase gates in the quantum fourier transform in terms of phase gates on one qubit and controlled not gates.

### Complexity & Resources

Each controlled phase two qubit gate implementation uses the following:
- 2 CX gates
- 3 Phase gates

It also has depth 4.

For the controlled phase three qubit gate, we use:
- 8 CX gates
- 7 Phase gates

Where the depth is now 10. 
Theoretically, there should be an optimization with at most CX gates. 
I will optimize it more later.

In [ ]:
def controlled_three_not(register):
    # the first three act as control qubits, the fourth is the target, and the fifth is a borrowed qubit
    quantum_circuit = QuantumCircuit(register, name=r"C^{3}X")

    quantum_circuit.ccx(register[4], register[2], register[3])
    quantum_circuit.ccx(register[0], register[1], register[4])
    quantum_circuit.ccx(register[4], register[2], register[3])
    quantum_circuit.ccx(register[0], register[1], register[4])

    return quantum_circuit

def controlled_phase(theta, register):
    # Creates a controlled phase gate using phase gates and cnot
    # Inputs are a quantum register and a theta angle
    # If there are more than three qubits, the first three act as control qubits, the fourth is the target, and the fifth is a borrowed qubit
    quantum_circuit = QuantumCircuit(register, name = f"CP({theta})")
    if len(register) == 2:
        quantum_circuit.cx(register[0], register[1])
        quantum_circuit.p(- theta / 2, register[1])
        quantum_circuit.cx(register[0],register[1])
        quantum_circuit.p(theta / 2, register[0])
        quantum_circuit.p(theta / 2, register[1])
    elif len(register) == 3:
        quantum_circuit.ccx(register[0], register[1], register[2])
        quantum_circuit.p(- theta / 2, register[2])
        quantum_circuit.ccx(register[0], register[1], register[2])
        quantum_circuit.p(theta / 2, register[2])
        phase = controlled_phase(theta / 2, register[:-1])
        quantum_circuit.compose(phase, qubits=register[:-1], inplace=True)
    elif len(register) == 5:
        CCCNOT = controlled_three_not(register)
        quantum_circuit.compose(CCCNOT, qubits=register, inplace=True)
        quantum_circuit.p(- theta / 2, register[3])
        quantum_circuit.compose(CCCNOT, qubits=register, inplace=True)
        quantum_circuit.p(theta / 2, register[3])
        phase = controlled_phase(theta / 2, register[:3])
        quantum_circuit.compose(phase, qubits=register[:3], inplace=True)
    else:
        raise Exception("Register not right size")

    return quantum_circuit

register = list(x_register) + list(z_register) + list(y_register)
phase = controlled_phase(np.pi / 4, register)

for x in range(4):
    for y in range(4):
        for z in range(2):
            start = x * 8 + z * 4 + y # right way
            start = (y)* 8 + z* 4 + x # little endian
            startState = Statevector.from_int(start, 2**5)
            endState = startState.evolve(phase)
            print(startState.inner(endState), x, z, y)

phase.draw()

(1+0j) 0 0 0
(1+0j) 0 1 0
(1+0j) 0 0 1
(1+0j) 0 1 1
(1+0j) 0 0 2
(1+0j) 0 1 2
(1+0j) 0 0 3
(1+0j) 0 1 3
(1+0j) 1 0 0
(1+0j) 1 1 0
(1+0j) 1 0 1
(1+0j) 1 1 1
(1+0j) 1 0 2
(1+0j) 1 1 2
(1+0j) 1 0 3
(1+0j) 1 1 3
(1+0j) 2 0 0
(1+0j) 2 1 0
(1+0j) 2 0 1
(1+0j) 2 1 1
(1+0j) 2 0 2
(1+0j) 2 1 2
(1+0j) 2 0 3
(1+0j) 2 1 3
(1+1.3877787807814457e-17j) 3 0 0
(1-1.3877787807814457e-17j) 3 1 0
(1+1.3877787807814457e-17j) 3 0 1
(0.7071067811865475+0.7071067811865476j) 3 1 1
(1+1.3877787807814457e-17j) 3 0 2
(1-1.3877787807814457e-17j) 3 1 2
(1+1.3877787807814457e-17j) 3 0 3
(0.7071067811865475+0.7071067811865476j) 3 1 3


»
0: ───────■─────────■────────────────────■─────────■──────■─────────────────»
          │         │                    │         │      │                 »
1: ───────■─────────■────────────────────■─────────■──────■─────────────────»
          │         │                    │         │    ┌─┴─┐   ┌──────────┐»
2: ──■────┼────■────┼───────────────■────┼────■────┼────┤ X ├───┤ P(-π/16) ├»
   ┌─┴─┐  │  ┌─┴─┐  │  ┌─────────┐┌─┴─┐  │  ┌─┴─┐  │  ┌─┴───┴──┐└──────────┘»
3: ┤ X ├──┼──┤ X ├──┼──┤ P(-π/8) ├┤ X ├──┼──┤ X ├──┼──┤ P(π/8) ├────────────»
   └─┬─┘┌─┴─┐└─┬─┘┌─┴─┐└─────────┘└─┬─┘┌─┴─┐└─┬─┘┌─┴─┐└────────┘            »
4: ──■──┤ X ├──■──┤ X ├─────────────■──┤ X ├──■──┤ X ├──────────────────────»
        └───┘     └───┘                └───┘     └───┘                      »
«                                    ┌─────────┐
«0: ──■───────■───────────────────■──┤ P(π/32) ├
«     │     ┌─┴─┐   ┌──────────┐┌─┴─┐├─────────┤
«1: ──■─────┤ X ├───┤ P(-π/32) ├┤ X ├┤ P(π/32) ├
«   ┌─┴─┐┌──┴───┴──┐└──────────┘└───┘└─────────┘
«2: ┤ X ├┤ P(π/16) ├────────────────────────────
«   └───┘└─────────┘                            
«3: ────────────────────────────────────────────
«                                               
«4: ────────────────────────────────────────────
«

We need a swap gate decomposed into controlled nots for the same reasoning

### Complexity & Resources

Each swap  gate implementation uses the following:
- 3 CX gates

It also has depth 2.

In [35]:
def swap(register):
    # Creates a swap gate using cnot gates
    # Inputs are a quantum register
    # Always uses first two qubits in register
    quantum_circuit = QuantumCircuit(register, name = "SWAP")    
    quantum_circuit.cx(register[0], register[1])
    quantum_circuit.cx(register[1], register[0])
    quantum_circuit.cx(register[0], register[1])

    return quantum_circuit

#swap(x_register).draw()

We next implement a quantum fourier transform (QFT) and its inverse in much the same way as the second problem session.
Notably, I have added a flag for whether the (QFT) swaps the qubits around after the computation.
This is to save CX gates when computing the QFT followed by its inverse.

### Complexity & Resources

Each QFT implementation on an $n$-qubit register uses the following:
- $n$ Hadamard gates
- $\frac{n^2-n}{2}$ controlled phase gates

When decomposed into just CX and 1-qubit gates, we get:
- $n$ Hadamard gates
- $\frac{3n^2-3n}{2}$ phase gates
- $n^2-n$ CX gates

Finally, the depth is $n$, with the inverse QFT having similar complexity and resources.

In [36]:
def quantum_fourier_transform(register):
    # Creates the quantum fourier transform on a given register
    # Inputs are a quantum register and a default boolean swap
    # needSwap set to True adds swap gates to the result to get the correct order
    # If we are immediately doing the inverse circuit, then it is an unnecessary swap
    # Saves gates to set needSwap = False

    n = len(register) # First get the number of qubits
    quantum_circuit = QuantumCircuit(register, name = "QFT")

    for j, qubit in enumerate(register):
        quantum_circuit.h(qubit)
        for k in range(j+1,n):
            theta = np.pi / (2**(k-j))
            phase = controlled_phase(theta, [register[k], qubit]).to_gate()
            quantum_circuit.compose(phase, qubits=[k,j],inplace=True)


    return quantum_circuit

def inverse_quantum_fourier_transform(register):
    # Creates the inverse quantum fourier transform on a given register
    # Inputs are a quantum register and a default boolean swap
    # needSwap set to True adds swap gates to the initial register to get the correct order
    # If we are immediately following the forward circuit, then it is an unnecessary swap
    # Saves gates to set needSwap = False
    n = len(register) # First get the number of qubits
    quantum_circuit = QuantumCircuit(register, name = "IQFT")
    
    for j, qubit in enumerate(register[::-1]):
        for k in range(n,n-j,-1):
            theta = -np.pi / (2**(k+j-n))
            phase = controlled_phase(theta, [register[k-1], qubit])
            quantum_circuit.compose(phase, qubits=[k-1,qubit],inplace=True)
        
        quantum_circuit.h(qubit)

    return quantum_circuit

Now that we have our necessary components, we can begin by implementing an adder.
The design for this adder comes from Thomas G. Draper's adder using the QFT, [first produced in 1998](https://arxiv.org/pdf/quant-ph/0008033).
The way it works is through the transform addition circuit implemented below.
Once the QFT is applied to the qubits representing one of our numbers, we perform phase rotations on the result controlled by the other number.
This is the same as adding before the QFT.
We then apply the inverse QFT to discover our result.

### Complexity & Resources

Each Transform Adder implementation on two $n$-qubit registers uses the following:
- $\frac{n^2+n}{2}$ controlled phase gates

When decomposed into just CX and 1-qubit gates, we get:
- $\frac{3n^2+3n}{2}$ phase gates
- $n^2+n $ CX gates

Finally, the depth is $n$.

In [37]:
def transform_adder(*summands):
    # Creates the transform adder circuit described in Draper's paper
    # Takes as input summands, which is a list of two registers
    # The first summand is added to the second in place
    # The second summand needs to be the result of a prior QFT for this to work
    # Any third possible summand is taken as a control qubit
    a_register = summands[0]
    b_register = summands[1]

    assert(len(a_register) <= len(b_register))

    quantum_circuit = QuantumCircuit(*summands, name="TADD")

    if len(summands) > 2:
        control = summands[2][0]
    
        for j, b in enumerate(b_register):
            for k, a in enumerate(a_register[j:]):
                theta = np.pi / (2**(k))
                phase = controlled_phase(theta, [a,control, b]).to_gate()
                quantum_circuit.compose(phase, qubits=[a,control, b], inplace=True)
    else:
        for j, b in enumerate(b_register):
            for k, a in enumerate(a_register[j:]):
                theta = np.pi / (2**(k))
                phase = controlled_phase(theta, [a,b])
                quantum_circuit.compose(phase, qubits=[a,b], inplace=True)

    return quantum_circuit

transform_adder(x_register, y_register, z_register).draw()

┌────────────────────────┐                           »
x_0: ┤0                       ├───────────────────────────»
     │                        │┌─────────────────────────┐»
x_1: ┤                        ├┤0                        ├»
     │                        ││                         │»
y_0: ┤2 CP(3.141592653589793) ├┤2                        ├»
     │                        ││  CP(1.5707963267948966) │»
y_1: ┤                        ├┤                         ├»
     │                        ││                         │»
  z: ┤1                       ├┤1                        ├»
     └────────────────────────┘└─────────────────────────┘»
«                               
«x_0: ──────────────────────────
«     ┌────────────────────────┐
«x_1: ┤0                       ├
«     │                        │
«y_0: ┤                        ├
«     │  CP(3.141592653589793) │
«y_1: ┤2                       ├
«     │                        │
«  z: ┤1                       ├
«     └────────────────────────┘

We are now ready to implement our first addition circuit.

### Complexity & Resources

Each Modulo Adder implementation on two $n$-qubit registers uses the following:
- 1 QFT
- 1 Transform Adder
- 1 Inverse QFT

When decomposed into just CX and 1-qubit gates, we get:
- $\frac{9n^2-3n}{2}$ phase gates
- $2n$ Hadamard gates
- $3n^2 - n$ CX gates

Finally, the depth is $3n$.

In [38]:
def modulo_addition(a_register, b_register, *control):
    # Implements addition modulo $2^d$, where $d$ was defined in our parameters
    # Takes in two registers as summands and their sum is written to the second register
    # Takes in an optional control qubit as well
    quantum_circuit = QuantumCircuit(a_register, b_register, *control, name="MADD")
    assert(len(a_register) == len(b_register))

    QFT = quantum_fourier_transform(b_register).to_gate()
    quantum_circuit.compose(QFT, qubits=b_register, inplace=True)

    TADD = transform_adder(a_register, b_register, *control).to_gate()
    quantum_circuit.compose(TADD, inplace=True)
    
    IQFT = inverse_quantum_fourier_transform(b_register).to_gate()
    quantum_circuit.compose(IQFT, qubits=b_register, inplace=True)

    return quantum_circuit

In [ ]:
def transform_multiplier(a_register, b_register, output_register, control_register):
    # Creates the a variant of the multiplication circuit described in Ruiz-Perez and Garcia-Escartin's paper
    # Takes as input four registers
    # The first register is multiplied by the second in the third
    # The third register needs to be the result of a prior QFT of 0 for this to work
    # Fourth register taken as a control qubit
    assert(len(a_register) == len(b_register))

    quantum_circuit = QuantumCircuit(a_register, b_register, output_register, control_register, name="TMUL")
    borrow = a_register[0]

    for j, a in enumerate(a_register[::-1]):
        for k, c in enumerate(output_register):
            for m, b in enumerate(b_register[k+j:]):
                register = [control_register[0], a, b, c, borrow]
                theta = np.pi / (2 ** m)
                phase = controlled_phase(theta, register).to_gate()
                quantum_circuit.compose(phase, qubits=register, inplace=True)
        borrow = a_register[-1]            

    return quantum_circuit
    
TMUL = transform_multiplier(x_register, y_register, output_register, z_register)
TMUL.draw()


┌────────────────────────┐┌─────────────────────────┐»
x_0: ┤4                       ├┤4                        ├»
     │                        ││                         │»
x_1: ┤1                       ├┤1                        ├»
     │                        ││                         │»
y_0: ┤2                       ├┤                         ├»
     │                        ││                         │»
y_1: ┤  CP(3.141592653589793) ├┤2 CP(1.5707963267948966) ├»
     │                        ││                         │»
w_0: ┤3                       ├┤3                        ├»
     │                        ││                         │»
w_1: ┤                        ├┤                         ├»
     │                        ││                         │»
  z: ┤0                       ├┤0                        ├»
     └────────────────────────┘└─────────────────────────┘»
«     ┌────────────────────────┐┌────────────────────────┐
«x_0: ┤4                       ├┤1                       ├
«     │                        ││                        │
«x_1: ┤1                       ├┤4                       ├
«     │                        ││                        │
«y_0: ┤                        ├┤                        ├
«     │                        ││                        │
«y_1: ┤2 CP(3.141592653589793) ├┤2 CP(3.141592653589793) ├
«     │                        ││                        │
«w_0: ┤                        ├┤3                       ├
«     │                        ││                        │
«w_1: ┤3                       ├┤                        ├
«     │                        ││                        │
«  z: ┤0                       ├┤0                       ├
«     └────────────────────────┘└────────────────────────┘

Before adding this to the full calculator circuit, we must first build two essentially bookkeeping circuits.
As Qiskit encodes its vectors with little-endian ordering, I need to convert little-endian to big-endian for my own ease.
The copy circuit just allows me to not modify the numbers in place.

### Complexity and Resources

The copy circuit on $n$-qubits is relatively simple when copying into an initial 0 state, using only:
- $n$ CX gates

The reverse endian circuit is a little more complicated, but it uses only:
- $3n$ CX gates

It has depth $3n$ and the copy circuit has depth $n$.

In [40]:
def copy_register(in_register, output, *control):
    # Copies the given input to an output initialized to 0
    # Takes in two registers
    # Optionally takes in a third qubit to act as control
    extra = False
    if len(control) == 1:
        quantum_circuit = QuantumCircuit(in_register, output, *control, name = "COPY")
        extra = True
    else:
        quantum_circuit = QuantumCircuit(in_register, output, name = "COPY")
    
    for j, x in enumerate(in_register):
        if extra:
            quantum_circuit.ccx(x,*control,output[j])
        else:
            quantum_circuit.cx(x,output[j])

    return quantum_circuit

def reverse_endian(register, *control):
    # Reverses the endian of how Qiskit handles states
    # Takes in a single register
    quantum_circuit = QuantumCircuit(register, *control, name = "REND")
    n = len(register)

    h = n // 2
    offset = n % 2

    for p, q in zip(register[:h], reversed(register[h+offset:])):
        quantum_circuit.compose(swap([p,q]), qubits=[p,q], inplace=True)

    return quantum_circuit

In [41]:
def QCalc(x_register, y_register, z_register, output_register):
    # Implements the calculator, at least the addition part

    quantum_circuit = QuantumCircuit(x_register, y_register, z_register, output_register, name = "QCALC")
    register = quantum_circuit.qubits

    REND = reverse_endian(register).to_gate()
    quantum_circuit.compose(REND, register, inplace=True)

    quantum_circuit.x(z_register[0]) # Makes it so that the addition happens when z = 0

    COPY = copy_register(x_register, output_register, z_register).to_gate()
    quantum_circuit.compose(COPY, qubits= list(x_register) + list(output_register) + list(z_register), inplace=True)

    MADD = modulo_addition(y_register, output_register, z_register).to_gate()
    quantum_circuit.compose(MADD, qubits= list(y_register) + list(output_register) + list(z_register), inplace=True)

    quantum_circuit.x(z_register[0])
    
    quantum_circuit.compose(REND, register, inplace=True)

    return quantum_circuit

mod = 2**d
dims = 2**(3*d+1)
QCALC = QCalc(x_register, y_register, z_register, output_register)

for a in range(mod):
    for b in range(mod):
        for z in range(2):
            start = a*(mod**2)*2 + b*mod*2 + z*mod
            startState = Statevector.from_int(start, dims)
            end = a*(mod**2)*2 + b*mod*2 + z*mod + ((a+b) % mod)
            endState = Statevector.from_int(end, dims)
            state = startState.evolve(QCALC)

            if not state.equiv(endState):
                if z == 1 and not state.equiv(state):
                    print(f"Something Broke {a,b,z}")

QCALC.draw()

┌───────┐     ┌───────┐              ┌───────┐
x_0: ┤0      ├─────┤0      ├──────────────┤0      ├
     │       │     │       │              │       │
x_1: ┤1      ├─────┤1      ├──────────────┤1      ├
     │       │     │       │┌───────┐     │       │
y_0: ┤2      ├─────┤       ├┤0      ├─────┤2      ├
     │       │     │       ││       │     │       │
y_1: ┤3 REND ├─────┤  COPY ├┤1      ├─────┤3 REND ├
     │       │┌───┐│       ││       │┌───┐│       │
  z: ┤4      ├┤ X ├┤4      ├┤4 MADD ├┤ X ├┤4      ├
     │       │└───┘│       ││       │└───┘│       │
w_0: ┤5      ├─────┤2      ├┤2      ├─────┤5      ├
     │       │     │       ││       │     │       │
w_1: ┤6      ├─────┤3      ├┤3      ├─────┤6      ├
     └───────┘     └───────┘└───────┘     └───────┘

In [55]:
def QCalc(x_register, y_register, z_register, output_register):
    # Implements the calculator, at least the addition part

    quantum_circuit = QuantumCircuit(x_register, y_register, z_register, output_register, name = "QCALC")
    register = quantum_circuit.qubits

    REND = reverse_endian(register).to_gate()
    quantum_circuit.compose(REND, register, inplace=True)

    quantum_circuit.x(z_register[0]) # Makes it so that the addition happens when z = 0

    COPY = copy_register(x_register, output_register, z_register).to_gate()
    quantum_circuit.compose(COPY, qubits= list(x_register) + list(output_register) + list(z_register), inplace=True)
    
    QFT = quantum_fourier_transform(output_register).to_gate()
    quantum_circuit.compose(QFT, qubits=output_register, inplace=True)

    #MADD = modulo_addition(y_register, output_register, z_register).to_gate()
    #quantum_circuit.compose(MADD, qubits= list(y_register) + list(output_register) + list(z_register), inplace=True)

    TADD = transform_adder(y_register, output_register, z_register).to_gate()
    quantum_circuit.compose(TADD, qubits= list(y_register) + list(output_register) + list(z_register), inplace=True)

    quantum_circuit.x(z_register[0])

    SMRD = reverse_endian(output_register).to_gate()
    quantum_circuit.compose(SMRD, output_register, inplace=True)

    TMUL = transform_multiplier(x_register, y_register, output_register, z_register).to_gate()
    quantum_circuit.compose(TMUL, qubits= register, inplace=True)

    quantum_circuit.compose(SMRD, output_register, inplace=True)

    IQFT = inverse_quantum_fourier_transform(output_register).to_gate()
    quantum_circuit.compose(IQFT, qubits=output_register, inplace=True)
    
    quantum_circuit.compose(REND, register, inplace=True)

    return quantum_circuit

mod = 2**d
dims = 2**(3*d+1)
QCALC = QCalc(x_register, y_register, z_register, output_register)

for a in range(mod):
    for b in range(mod):
        for z in range(2):
            start = a*(mod**2)*2 + b*mod*2 + z*mod
            startState = Statevector.from_int(start, dims)
            operation = (a + b) % mod if z == 0 else (a*b) % mod
            end = a*(mod**2)*2 + b*mod*2 + z*mod + operation
            endState = Statevector.from_int(end, dims)
            state = startState.evolve(QCALC)

            if not state.equiv(endState):
                print(f"Something Broke {a,b,z}")
                print(state.inner(endState))
                #print(state)

QCALC.draw()

Something Broke (1, 1, 0)
(0.5+0j)
Something Broke (1, 1, 1)
(0.5+0j)
Something Broke (1, 3, 0)
(0.5+0j)
Something Broke (1, 3, 1)
(0.5+0j)
Something Broke (3, 1, 0)
(0.5+0j)
Something Broke (3, 1, 1)
(0.5+0j)
Something Broke (3, 3, 0)
(0.5+0j)
Something Broke (3, 3, 1)
(0.5+0j)


┌───────┐     ┌───────┐                          ┌───────┐         »
x_0: ┤0      ├─────┤0      ├──────────────────────────┤0      ├─────────»
     │       │     │       │                          │       │         »
x_1: ┤1      ├─────┤1      ├──────────────────────────┤1      ├─────────»
     │       │     │       │        ┌───────┐         │       │         »
y_0: ┤2      ├─────┤       ├────────┤0      ├─────────┤2      ├─────────»
     │       │     │       │        │       │         │       │         »
y_1: ┤3 REND ├─────┤  COPY ├────────┤1      ├─────────┤3 TMUL ├─────────»
     │       │┌───┐│       │        │       │  ┌───┐  │       │         »
  z: ┤4      ├┤ X ├┤4      ├────────┤4 TADD ├──┤ X ├──┤4      ├─────────»
     │       │└───┘│       │┌──────┐│       │┌─┴───┴─┐│       │┌───────┐»
w_0: ┤5      ├─────┤2      ├┤0     ├┤2      ├┤0      ├┤5      ├┤0      ├»
     │       │     │       ││  QFT ││       ││  REND ││       ││  REND │»
w_1: ┤6      ├─────┤3      ├┤1     ├┤3      ├┤1      ├┤6      ├┤1      ├»
     └───────┘     └───────┘└──────┘└───────┘└───────┘└───────┘└───────┘»
«              ┌───────┐
«x_0: ─────────┤0      ├
«              │       │
«x_1: ─────────┤1      ├
«              │       │
«y_0: ─────────┤2      ├
«              │       │
«y_1: ─────────┤3 REND ├
«              │       │
«  z: ─────────┤4      ├
«     ┌───────┐│       │
«w_0: ┤0      ├┤5      ├
«     │  IQFT ││       │
«w_1: ┤1      ├┤6      ├
«     └───────┘└───────┘

In [62]:
def modulo_multiplier(x_register, y_register, z_register, output_register):
    # Implements the calculator, at least the addition part

    quantum_circuit = QuantumCircuit(x_register, y_register, z_register, output_register, name = "QCALC")
    register = quantum_circuit.qubits

    REND = reverse_endian(register).to_gate()
    quantum_circuit.compose(REND, register, inplace=True)
    
    QFT = quantum_fourier_transform(output_register).to_gate()
    quantum_circuit.compose(QFT, qubits=output_register, inplace=True)

    SMRD = reverse_endian(output_register)
    quantum_circuit.compose(SMRD, output_register, inplace=True)

    TMUL = transform_multiplier(x_register, y_register, output_register, z_register).to_gate()
    quantum_circuit.compose(TMUL, qubits= register, inplace=True)

    quantum_circuit.compose(SMRD, output_register, inplace=True)

    IQFT = inverse_quantum_fourier_transform(output_register).to_gate()
    quantum_circuit.compose(IQFT, qubits=output_register, inplace=True)
    
    quantum_circuit.compose(REND, register, inplace=True)

    return quantum_circuit

mod = 2**d
dims = 2**(3*d+1)
MMUL = modulo_multiplier(x_register, y_register, z_register, output_register)

for a in range(mod):
    for b in range(mod):
        z == 1
        start = a*(mod**2)*2 + b*mod*2 + z*mod
        startState = Statevector.from_int(start, dims)
        operation = (a + b) % mod if z == 0 else (a*b) % mod
        end = a*(mod**2)*2 + b*mod*2 + z*mod + operation
        endState = Statevector.from_int(end, dims)
        state = startState.evolve(MMUL)

        if not state.equiv(endState):
            print(f"Something Broke {a,b,z}")
            print(state.inner(endState))
            #print(state)

MMUL.draw()

Something Broke (1, 1, 1)
(0.5+0j)
Something Broke (1, 3, 1)
(0.5+0j)
Something Broke (3, 1, 1)
(0.5+0j)
Something Broke (3, 3, 1)
(0.5+0j)


┌───────┐                       ┌───────┐                        ┌───────┐
x_0: ┤0      ├───────────────────────┤0      ├────────────────────────┤0      ├
     │       │                       │       │                        │       │
x_1: ┤1      ├───────────────────────┤1      ├────────────────────────┤1      ├
     │       │                       │       │                        │       │
y_0: ┤2      ├───────────────────────┤2      ├────────────────────────┤2      ├
     │       │                       │       │                        │       │
y_1: ┤3 REND ├───────────────────────┤3 TMUL ├────────────────────────┤3 REND ├
     │       │                       │       │                        │       │
  z: ┤4      ├───────────────────────┤4      ├────────────────────────┤4      ├
     │       │┌──────┐     ┌───┐     │       │     ┌───┐     ┌───────┐│       │
w_0: ┤5      ├┤0     ├──■──┤ X ├──■──┤5      ├──■──┤ X ├──■──┤0      ├┤5      ├
     │       ││  QFT │┌─┴─┐└─┬─┘┌─┴─┐│       │┌─┴─┐└─┬─┘┌─┴─┐│  IQFT ││       │
w_1: ┤6      ├┤1     ├┤ X ├──■──┤ X ├┤6      ├┤ X ├──■──┤ X ├┤1      ├┤6      ├
     └───────┘└──────┘└───┘     └───┘└───────┘└───┘     └───┘└───────┘└───────┘